In [ ]:
uff_subtracted_dataframes = [] #create empty list to store buffer subtracted data
amideI_dataframes = []  #create empty list to store Amide I Area normalized data

source_file = eval(input("""Enter 1 to import all data from a single csv (with headers), or 2 for individual data and buffer files (no headers) """))

if source_file == 1: 
    source_file = True
else: source_file = False
    
if source_file:
    raw_data_filename, directory = select_file()

    rawData_df, raw_data_filenames = create_df_from_single_file(raw_data_filename, directory)  #Creates the raw signal dataframe of buffer and each protein through the loop   
    print(raw_data_filenames)
    print(rawData_df.head(3))
    
else:
    raw_data_filenames, directory = select_files() #Calls select_files to read and pass back the filenames of interest in a list
    
    rawData_df, raw_data_filenames = create_df_from_multiple_files(raw_data_filenames, directory)  #Creates the raw signal dataframe of buffer and each protein through the loop   
    
num_files = len(raw_data_filenames)-2 #length excludes buffer file
initial_x_df = rawData_df.iloc[:, 0:2].copy()  #copies x-axis and buffer data for loop dataframe 


for i in range(num_files): #Loops through all selected files 
    raw_data_file = raw_data_filenames[i+2] #Loops through filename 1 - range to create buffer and protein list

    raw_data_file_df1 = rawData_df[[raw_data_file]].copy()  #Creates the raw signal dataframe of buffer and each protein through the loop   

    raw_data_file_df2 = pd.concat([initial_x_df, raw_data_file_df1], axis=1)

    protein = raw_data_file_df2.columns[2] #Gets column name of current spectra being processed
    print('Currently processing file: ' + protein)    

    subtr_df = buffer_subtract(raw_data_file_df2).copy() #adds buffer subtracted spectra to df  

    
    full_x_df = subtr_df.iloc[:, [0]].copy()  #copies x-axis data for building buffer subtracted dataframe 
    buffsub_col = subtr_df[[protein + '_subtracted']] #Extract only the buffer subtracted column    
    buff_subtracted_dataframes.append(buffsub_col)
    
    crop_df = crop(subtr_df) #truncates data to amide I region
    crop_x_df = crop_df.iloc[:, [0]].copy()  #copies x-axis data for building buffer subtracted dataframe 
    amideI_col = crop_df[[protein + '_subtracted']]
    amideI_dataframes.append(amideI_col)

    
all_subtracted_df = pd.concat(buff_subtracted_dataframes, axis=1)    
all_subtracted_df = pd.concat([full_x_df, all_subtracted_df], axis=1)
all_subtracted_df.plot(x='freq', figsize=(14,10))

plt.savefig(directory +'subtr'+'.png')
plt.show()
plt.close()

all_amideI_df = pd.concat(amideI_dataframes, axis=1)    
all_amideI_df = pd.concat([crop_x_df, all_amideI_df], axis=1)

all_subtracted_df.to_csv(directory + '/' + 'BufferSubtracted'+'.csv', index=False)
all_amideI_df.to_csv(directory + '/' + 'Amide_I_Buffer_Subtracted'+'.csv', index=False)

print('\nDone!')
